# Clean Data

### Imports

In [152]:
import numpy as np
import pandas as pd


## Clean NFL Combine Data

In [153]:
combine_file = r'data\nfl_combine_1987_2020.csv'

df_raw_combine = pd.read_csv(combine_file)

df_raw_combine.head()

# Keep raw data import for reference - build copy to modify
df_combine = df_raw_combine

# Drop dummy '0' column and Wonderlic scores data
combine_cols_to_drop = ['Unnamed: 0', 'Wonderlic']
df_combine.drop(columns=combine_cols_to_drop, inplace=True)

# Clean column headers
df_combine.columns = df_combine.columns.str.lower()
df_combine.rename(columns={'college':'school'}, inplace=True)

# Clean school names
df_combine['school'] = df_combine['school'].str.strip()

# Drop years prior to 2000 (no draft data)
print(df_combine.shape)
df_combine.drop(df_combine[df_combine['year']<2000].index, inplace=True)
print('Cleaned combine size: ', df_combine.shape)


(12808, 15)
Cleaned combine size:  (8465, 15)


## Clean NFL Draft Data

In [154]:
draft_file = r'data\espn_draft_history_2000_2021_cleaned.csv'
df_raw_draft = pd.read_csv(draft_file)

# Keep raw data import for reference - build copy to modify
df_draft = df_raw_draft

# Clean column headers
df_draft.columns = df_draft.columns.str.lower()

# Clean school names
df_draft['school'] = df_draft['school'].str.strip()


In [155]:
# combine results by position
df_combine['pos'].value_counts()


WR     1095
CB      864
RB      700
DE      657
DT      647
OT      618
OLB     586
OG      528
TE      471
QB      470
ILB     350
FS      347
SS      298
C       253
FB      175
P       108
K        92
LB       91
S        53
LS       22
EDG      22
OL       10
DL        4
DB        3
NT        1
Name: pos, dtype: int64

### Are there duplicated names?

In [156]:
df_combine['name'].value_counts(sort='descending').head(10)

Brandon Williams    5
Chris Brown         5
Brian Allen         4
Mike Williams       4
Chris Jones         4
Michael Bennett     3
Josh Harris         3
Travis Wilson       3
Steve Smith         3
Andre Smith         3
Name: name, dtype: int64

### Answer: Yes

So we cannot simply join the 2 datasets on player 'name' columns. Need to also join on
college and year.


## Do college names match in both datasets?

In [157]:
draft_school = pd.DataFrame(df_draft['school'].unique()).rename(columns={0:'school'})
draft_school['source'] = 'draft'
combine_school = pd.DataFrame(df_combine['school'].unique()).rename(columns={0:'school'})
combine_school['source'] = 'combine'
print(type(combine_school))
print(combine_school.head())

schools = draft_school.merge(combine_school, on='school', how='outer',
                             suffixes=['_draft', '_combine']).sort_values(by='school')

# List all cases with mismatches
na_mask = schools.isna().any(axis=1)
print(schools[na_mask])

<class 'pandas.core.frame.DataFrame'>
               school   source
0      Boston College  combine
1          Texas Tech  combine
2  Jackson State (MS)  combine
3      South Carolina  combine
4         Wake Forest  combine
                       school source_draft source_combine
202         Abilene Christian        draft            NaN
381    Abilene Christian (TX)          NaN        combine
445          Adams State (CO)          NaN        combine
345        Alabama-Birmingham          NaN        combine
200                    Albany        draft            NaN
..                        ...          ...            ...
225             Winston-Salem        draft            NaN
348  Winston-Salem State (NC)          NaN        combine
280          Youngstown State        draft            NaN
325     Youngstown State (OH)          NaN        combine
103                       NaN        draft        combine

[315 rows x 3 columns]


### So we see that the 'combine' dataset frequently has the state appended to the school name;
Ex: "Abilene Christian (TX)". Remove these from school names, with the exception of "Miami (OH).

In [158]:
df_combine['school'] = df_combine['school'].str.replace('Miami (OH)', 'Miami - OH')

print(df_combine['school'].head())
regex_replace_parens = r'\([^)]*[a-zA-Z][^)]*\)'
df_combine['school'] = df_combine['school'].str.replace(regex_replace_parens, '', regex=True)

print(df_combine['school'].head())

# pd.DataFrame(df_combine['school'].unique()).to_clipboard()
#
# schools2 = df_draft.merge(df_combine,
#                           on='school', how='outer',
#                           suffixes=['_draft', '_combine']).sort_values(by='school')
#
# na_mask = schools2.isna().any(axis=1)
# schools2[na_mask]
# pd.concat([df_combine['school'], df_draft['school']], join='outer')

#df_combine.merge(df_draft, how='left', on='school')

4343         Boston College
4344             Texas Tech
4345     Jackson State (MS)
4346         South Carolina
4347            Wake Forest
4348               Syracuse
4349               Colorado
4350               Arkansas
4351               Oklahoma
4352    Arkansas-Pine Bluff
4353      Mississippi State
4354          West Virginia
4355          Marshall (WV)
4356               Colorado
4357                Arizona
4358             Miami (FL)
4359                   Iowa
4360               Michigan
4361             Iowa State
4362             Utah State
Name: school, dtype: object
4343         Boston College
4344             Texas Tech
4345         Jackson State 
4346         South Carolina
4347            Wake Forest
4348               Syracuse
4349               Colorado
4350               Arkansas
4351               Oklahoma
4352    Arkansas-Pine Bluff
4353      Mississippi State
4354          West Virginia
4355              Marshall 
4356               Colorado
4357                

C:\Users\mitch\AppData\Local\Temp/ipykernel_38680/4032306494.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_combine['school'] = df_combine['school'].str.replace('Miami (OH)', 'Miami - OH')


## Standardize player names between datasets
Player names in the "Draft" dataset include suffixes including "Jr., II, III, IV", but these are NOT included in the "combine" dataset.

Standardize player names between datasets by removing these values from the "Draft" dataset.

In [159]:
regex_suffixes_to_remove = r'Jr\.$|III$|IIII$|IV$|, Jr.$'
df_draft['name'] = df_draft['name'].str.replace(regex_suffixes_to_remove, '', regex=True)

# df_draft['name'].to_clipboard()



Merge the Draft and NFL Combine datasets

In [160]:
df_merged = df_combine.merge(df_draft, how='left',
                             on=['name', 'school', 'year'])

df_merged.head(10)

# df_merged.to_clipboard()

,year,name,school,pos,height (in),weight (lbs),hand size (in),arm length (in),40 yard,bench press,vert leap (in),broad jump (in),shuttle,3cone,60yd shuttle,round,pk(ovr),team,position
0,2000,Darnell Alford,Boston College,OT,76.0,334,10.50,34.00,5.58,23.0,25.0,94.0,4.98,8.48,NaN,6.0,22(188),Chiefs,OT
1,2000,Kyle Allamon,Texas Tech,TE,74.5,253,9.88,31.00,5.01,NaN,29.0,104.0,4.49,7.29,12.47,NaN,NaN,NaN,NaN
2,2000,Rashard Anderson,Jackson State,CB,74.4,206,10.00,33.00,4.59,NaN,34.0,123.0,4.15,7.18,11.72,NaN,NaN,NaN,NaN
3,2000,Corey Atkins,South Carolina,OLB,72.4,237,9.75,31.88,4.76,21.0,31.0,112.0,4.39,7.96,11.87,NaN,NaN,NaN,NaN
4,2000,Reggie Austin,Wake Forest,CB,69.4,175,8.50,30.00,4.46,17.0,35.0,119.0,4.14,7.03,NaN,4.0,31(125),Bears,DB
5,2000,Mark Baniewicz,Syracuse,OT,77.9,312,9.25,32.75,5.36,20.0,28.0,96.0,4.73,7.72,NaN,7.0,41(247),Jaguars,OT
6,2000,Rashidi Barnes,Colorado,FS,71.6,208,9.50,32.00,4.63,10.0,35.0,114.0,4.32,6.92,11.35,7.0,19(225),Browns,DB
7,2000,David Barrett,Arkansas,CB,69.9,199,10.00,31.13,4.48,16.0,37.5,116.0,4.04,6.81,10.77,4.0,8(102),Cardinals,DB
8,2000,William Bartee,Oklahoma,CB,73.0,192,NaN,NaN,4.43,NaN,38.5,124.0,3.99,6.70,NaN,2.0,23(54),Chiefs,DB
9,2000,Terrance Beadles,Arkansas-Pine Bluff,OG,74.9,312,10.13,33.00,5.21,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Investigate merged data

In [161]:
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8465 entries, 0 to 8464
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             8465 non-null   int64  
 1   name             8465 non-null   object 
 2   school           8462 non-null   object 
 3   pos              8465 non-null   object 
 4   height (in)      8465 non-null   float64
 5   weight (lbs)     8465 non-null   int64  
 6   hand size (in)   7639 non-null   float64
 7   arm length (in)  7638 non-null   float64
 8   40 yard          7922 non-null   float64
 9   bench press      6603 non-null   float64
 10  vert leap (in)   7503 non-null   float64
 11  broad jump (in)  7400 non-null   float64
 12  shuttle          6929 non-null   float64
 13  3cone            6827 non-null   float64
 14  60yd shuttle     1370 non-null   float64
 15  round            3612 non-null   float64
 16  pk(ovr)          3612 non-null   object 
 17  team          